In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns=999
train=pd.read_csv('../input/DontGetKicked/training.csv')
train
test=pd.read_csv('../input/DontGetKicked/test.csv')
test

In [ ]:
all_data=pd.concat([train,test])
all_data

In [ ]:
all_data['PurchDate']=pd.to_datetime(all_data['PurchDate'])
all_data['Month']=all_data['PurchDate'].dt.month
all_data['Year']=all_data['PurchDate'].dt.year
all_data['Day']=all_data['PurchDate'].dt.day
all_data

In [ ]:
all_data2=all_data.drop(columns=['PurchDate','RefId','IsBadBuy'])
all_data2

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
c=all_data2.columns[all_data2.dtypes==object] #조건 부여 옵션 
for i in c:
    all_data2[i]=le.fit_transform(list(all_data2[i])) #먼저 리스트처리 해주기, list만나면 형식이 카테고리화 된다. 우리 모델이 학습을 할 수 있음. 결측치도 다 의미있음! 
    #eg.보험데이터의 경우, 공란으로 둔 설문이 있으면, 데이터 수집 시 결측치로 나오겠지? 근데 이것도 정보로 체크해서 받아들임, 이것도 학습해야해,그래서 카테고리화 시켜준다. 
    print(i)

In [ ]:
all_data2

In [ ]:
all_data2=all_data2.fillna(-1) 
# 결측치는 값이 비어있는 느낌에 가까움. 그 값 자체가 0인데, 값이 있어야 하는데 없는거랑은 다름. 결측치를 -1로 넣어버리면,결측치를 뭔가 이상한 값, 다른 값이라고 인식하게 된다.원래 들어가있는 데이터들과는 다른 느낌으로 값을 치환해줘야 한다. 대부분 값이 >0이라서 음수로 넣어줘버리자, 겹치지 않도록 , 평균은 오히려 겹치는 효과때문에


In [ ]:
train2=all_data2[:len(train)] # 맨 왼쪽부터 ㅌ레인 셋 갯수까지만
test2=all_data2[len(train):] #트레인 개수 제외하고 나머지를 들고 올게

트리모델에는 xgboost와 같은 boosting model.randomforest 라인들이 있다. 하나의 나무자체가 모델로 가정, 이걸 100개의 나무로 만들어서 종합해서 나온게 랜덤 포레스트임. 랜덤하게 나무를 만든다.  칼럼또한 랜덤하게 뽑아간다. 학습 시 나무마다 다양하게 학습시키려고 랜덤하게~
*ensemble: 나무들의 학습 방식을 다양한 패턴에 대한 데이터에 대처할 수 있게 해줌
항상 같은 데이터만 하면 모델이 적응을 하기 때문. 만들수록 성능 좋아줌. 

그 외 자주사용되는 모델
머신러닝 - catboost/lgbm/등등 
약점 보완용으로 -선형모델(거리기반으로 학습),,,,,
딥러닝 모델을 딥러닝에서 들고온다. + 트리모델 직접 쌓을 줄 알아야는 부분 

In [ ]:
#부스팅 모델끼리 서로 참고할 수 있도록 ㄷ와줌, 벗트 과적합 경계할 필요, 새로운데이터에대해 대응 X 
#만약에 난도 좀 있으면, xgboost 옵션/ 하이퍼파리미터를 조절해야지 안 밀린다. 옵션을 통해 개선필요

from xgboost import XGBClassifier
xg=XGBClassifier()
xg.fit(train2,train['IsBadBuy'])
result=xg.predict_proba(test2)
result

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_jobs=-1)
# rf.fit(train2, train['IsBadBuy']) 
# result = rf.predict_proba(test2)
# result

In [ ]:
sub=pd.read_csv('../input/DontGetKicked/example_entry.csv')
sub
sub['IsBadBuy']=result[:,1]
sub

In [ ]:
sub.to_csv('kickit4.csv',index=0)